<a href="https://colab.research.google.com/github/ganlongnz/Algorithm/blob/main/SentimentAnalysis_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!ls

sample_data


In [5]:
import torch
torch.cuda.is_available()

True

In [6]:
# check allocated GPU type
!nvidia-smi

Sun Sep  3 08:37:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
!pip install datasets transformers huggingface_hub

In [8]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [9]:
from datasets import load_dataset
imdb = load_dataset("imdb")

In [10]:
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [12]:
# Prepare contextualized embedding vectors for the train and test data sets.
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [13]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
# Import BERT.
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [16]:
from huggingface_hub import notebook_login
notebook_login()

In [18]:
!pip install -U transformers

In [15]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 5.1 MB/s eta 0:00:00


In [1]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.32.1', '0.22.0')

In [18]:
# Fine-tune model. Set hyperparameters.
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-sentiment-model-3000-samples_v1"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [19]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=376, training_loss=0.31237677310375456, metrics={'train_runtime': 289.1229, 'train_samples_per_second': 20.752, 'train_steps_per_second': 1.3, 'total_flos': 785643443397696.0, 'train_loss': 0.31237677310375456, 'epoch': 2.0})

In [20]:
trainer.evaluate()

<ipython-input-15-2457e8bb4065>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 0.3193594217300415,
 'eval_accuracy': 0.87,
 'eval_f1': 0.8712871287128714,
 'eval_runtime': 6.0207,
 'eval_samples_per_second': 49.828,
 'eval_steps_per_second': 3.156,
 'epoch': 2.0}

In [21]:
trainer.push_to_hub()

'https://huggingface.co/ganlongnz/finetuning-sentiment-model-3000-samples_v1/tree/main/'

In [25]:
from transformers import pipeline

sentiment_model = pipeline(model="ganlongnz/finetuning-sentiment-model-3000-samples_v1")
sentiment_model(["ABC is the rare movie that succeeds on almost every level.", "This movie sucks!"])

[{'label': 'LABEL_1', 'score': 0.944845974445343},
 {'label': 'LABEL_0', 'score': 0.964722216129303}]